<a href="https://colab.research.google.com/github/julwdo/NLP-project/blob/main/NLP_Project_JW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-17-jdk-headless -qq > /dev/null # OpenJDK 17
!wget --show-progress https://dlcdn.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz # Apache Spark 3.5.6 with Hadoop 3 support
!tar xf spark-3.5.6-bin-hadoop3.tgz
!pip install -q findspark
!pip install -q spark-nlp==6.1.2
!pip install -q xgboost==3.0.4
!pip install -q --upgrade pyspark==3.5.6

--2025-09-22 16:09:17--  https://dlcdn.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400923510 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.6-bin-hadoop3.tgz’

spark-3.5.6-bin-had 100%[===================>] 382.35M   160MB/s    in 2.4s    

2025-09-22 16:09:41 (160 MB/s) - ‘spark-3.5.6-bin-hadoop3.tgz’ saved [400923510/400923510]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.5.6-bin-hadoop3'

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("BotDetection")
    .master("local[*]")
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:6.1.2")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)
spark

In [4]:
from google.colab import auth
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
from nltk.sentiment import SentimentIntensityAnalyzer
import math
from collections import Counter
from pyspark.sql import Window
from pyspark.sql.types import StringType, NumericType, BooleanType, ArrayType, FloatType
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import XlmRoBertaSentenceEmbeddings
from pyspark.ml import Pipeline

In [5]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [6]:
auth.authenticate_user()

In [7]:
#!gcloud init

In [8]:
bucket_name = "twibot-22"
file_names = ["user.jsonl", "label.csv", "tweet_0.jsonl"]

for file_name in file_names:
    local_path = f"/content/{file_name}"
    if not os.path.exists(local_path):
        !gsutil cp gs://{bucket_name}/{file_name} {local_path}
    else:
        print(f"{file_name} already exists locally, skipping download.")

Copying gs://twibot-22/user.jsonl...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][745.4 MiB/745.4 MiB]   32.4 MiB/s                                   
Operation completed over 1 objects/745.4 MiB.                                    
Copying gs://twibot-22/label.csv...
/ [1 files][ 20.6 MiB/ 20.6 MiB]                                                
Operation completed over 1 objects/20.6 MiB.                                     
Copying gs://twibot-22/tweet_0.jsonl...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [1 files][ 11.1 GiB/ 11.1 GiB]  116.4 MiB/s                          

In [9]:
users = spark.read.json(f"/content/user.jsonl")
#users.printSchema()

In [10]:
#users.show(5, truncate=False)

In [11]:
users_selected = users.select(
    F.col("id"),
    F.col("name"),
    F.col("username"),
    F.col("created_at"),
    F.col("description"),
    F.col("url"),
    F.col("entities.description.cashtags"),
    F.col("entities.description.hashtags"),
    F.col("entities.description.mentions"),
    F.col("entities.description.urls"),
    F.col("location"),
    F.col("pinned_tweet_id"),
    F.col("profile_image_url"),
    F.col("protected"),
    F.col("public_metrics.followers_count"),
    F.col("public_metrics.following_count"),
    F.col("public_metrics.listed_count"),
    F.col("public_metrics.tweet_count"),
    F.col("verified")
    )

In [12]:
#users_selected.printSchema()

In [13]:
#users_selected.show(5, truncate=False)

In [14]:
labels = spark.read.csv(f"/content/label.csv", header=True, inferSchema=True)

In [15]:
labels.show(5, truncate=False)

+--------------------+-----+
|id                  |label|
+--------------------+-----+
|u1217628182611927040|human|
|u2664730894         |human|
|u1266703520205549568|human|
|u1089159225148882949|human|
|u36741729           |bot  |
+--------------------+-----+
only showing top 5 rows



In [16]:
users_labeled = users_selected.join(labels, users_selected.id == labels.id, "left").drop(labels.id)

In [17]:
#users_labeled.show(5, truncate=False)

In [18]:
#print('Summary of missing values:')
#users_labeled.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in users_labeled.columns]).show()

In [19]:
sia = SentimentIntensityAnalyzer()

def vader_sentiment(text):
  return sia.polarity_scores(text)["compound"]

vader_udf = F.udf(vader_sentiment, FloatType())

In [20]:
def shannon_entropy(string):
    if string.strip() == "":
        return 0.0
    counts = Counter(string)
    length = len(string)
    return -sum((count/length) * math.log2(count/length) for count in counts.values())

entropy_udf = F.udf(shannon_entropy, FloatType())

In [21]:
now = F.current_timestamp()

In [22]:
user_features = users_labeled.select(
    F.col("id"),
    F.length(F.col("name")).alias("name_length"),
    F.length(F.col("username")).alias("username_length"),
    (F.length(F.col("username")) / F.greatest(F.length(F.col("name")), F.lit(1))).alias("username_name_length_ratio"),
    F.regexp_replace(F.regexp_replace(F.regexp_replace(F.col("description"), r"https?://t\.co/\S+", "<URL>"), r"(?<=^|\s)@\w+", "<USER>"), r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "<EMAIL>").alias("description"),
    F.length(F.col("description")).alias("description_length"),
    F.when(F.col("name") == "", False).otherwise(True).alias("has_name"),
    F.when(F.col("username") == "", False).otherwise(True).alias("has_username"),
    F.when(F.col("description") == "", False).otherwise(True).alias("has_description"),
    F.when(F.col("url") == "", False).otherwise(True).alias("has_url"),
    F.when(F.col("location").isNull() | (F.col("location") == ""), False).otherwise(True).alias("has_location"),
    F.when(F.col("pinned_tweet_id").isNull(), False).otherwise(True).alias("has_pinned_tweet"),
    F.col("name").rlike("(?i)\\bbot\\b").alias("has_bot_word_in_name"),
    F.col("description").rlike("(?i)\\bbot\\b").alias("has_bot_word_in_description"),
    (F.length(F.regexp_replace(F.col("name"), "[^\\d]", "")) / F.greatest(F.length(F.col("name")), F.lit(1))).alias("ratio_digits_in_name"),
    (F.length(F.regexp_replace(F.col("username"), "[^\\d]", "")) / F.greatest(F.length(F.col("username")), F.lit(1))).alias("ratio_digits_in_username"),
    (F.length(F.regexp_replace(F.col("description"), "[^\\d]", "")) / F.greatest(F.length(F.col("description")), F.lit(1))).alias("ratio_digits_in_description"),
    (F.length(F.regexp_replace(F.col("name"), "[A-Za-z0-9 ]", "")) / F.greatest(F.length(F.col("name")), F.lit(1))).alias("ratio_special_chars_in_name"),
    (F.length(F.regexp_replace(F.col("username"), "[A-Za-z0-9 ]", "")) / F.greatest(F.length(F.col("username")), F.lit(1))).alias("ratio_special_chars_in_username"),
    (F.length(F.regexp_replace(F.col("description"), "[A-Za-z0-9 ]", "")) / F.greatest(F.length(F.col("description")), F.lit(1))).alias("ratio_special_chars_in_description"),
    (F.length(F.regexp_replace(F.col("name"), "[^A-Z]", "")) / F.greatest(F.length(F.regexp_replace(F.col("name"), "[^a-z]", "")), F.lit(1))).alias("name_upper_to_lower_ratio"),
    (F.length(F.regexp_replace(F.col("username"), "[^A-Z]", "")) / F.greatest(F.length(F.regexp_replace(F.col("username"), "[^a-z]", "")), F.lit(1))).alias("username_upper_to_lower_ratio"),
    entropy_udf(F.col("name")).alias("name_entropy"),
    entropy_udf(F.col("username")).alias("username_entropy"),
    (F.levenshtein(F.col("username"), F.col("name")) / F.greatest(F.length(F.col("username")), F.length(F.col("name")), F.lit(1))).alias("username_name_levenshtein"),
    vader_udf(F.col("description")).alias("description_sentiment"),
    F.when(F.col("cashtags").isNotNull(), F.size(F.col("cashtags"))).otherwise(F.lit(0)).alias("cashtag_in_description_count"),
    F.when(F.col("hashtags").isNotNull(), F.size(F.col("hashtags"))).otherwise(F.lit(0)).alias("hashtag_in_description_count"),
    F.when(F.col("mentions").isNotNull(), F.size(F.col("mentions"))).otherwise(F.lit(0)).alias("mention_in_description_count"),
    F.when(F.col("urls").isNotNull(), F.size(F.col("urls"))).otherwise(F.lit(0)).alias("url_in_description_count"),
    F.col("protected").alias("is_protected"),
    F.col("verified").alias("is_verified"),
    (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at"))).alias("account_age_seconds"),
#    F.col("followers_count"),
#    F.col("following_count"),
#    F.col("listed_count"),
#    F.col("tweet_count"),
    (F.col("followers_count") / F.greatest(F.col("following_count"), F.lit(1))).alias("followers_over_following"),
    (2 * F.col("followers_count") / F.greatest(F.col("following_count"), F.lit(1))).alias("double_followers_over_following"),
    (F.col("following_count") / F.greatest(F.col("followers_count"), F.lit(1))).alias("following_over_followers"),
    (F.col("following_count") / F.greatest(F.col("followers_count") ** 2, F.lit(1))).alias("following_over_followers_squared"),
    (F.col("following_count") / F.greatest(F.col("followers_count") + F.col("following_count"), F.lit(1))).alias("following_over_total_connections"),
    (F.col("listed_count") / F.greatest(F.col("followers_count"), F.lit(1))).alias("listed_over_followers"),
    (F.col("tweet_count") / F.greatest(F.col("followers_count"), F.lit(1))).alias("tweets_over_followers"),
    (F.col("listed_count") / F.greatest(F.col("tweet_count"), F.lit(1))).alias("listed_over_tweets"),
    (F.col("followers_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("follower_rate"),
    (F.col("following_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("following_rate"),
    (F.col("listed_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("listed_rate"),
    (F.col("tweet_count") / (F.unix_timestamp(now) - F.unix_timestamp(F.to_timestamp("created_at")))).alias("tweet_rate"),
    F.col("label")
    )

In [23]:
#user_features.show(5, truncate=False)

In [24]:
tweets = spark.read.json(f"/content/tweet_0.jsonl")
#tweets.printSchema()

In [25]:
#tweets.show(5, truncate=False)

In [26]:
tweets_selected = tweets.select(
    F.col("id"),
    F.regexp_replace(F.regexp_replace(F.regexp_replace(F.col("text"), r"https?://t\.co/\S+", "<URL>"), r"(?<=^|\s)@\w+", "<USER>"), r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "<EMAIL>").alias("text"),
    F.concat(F.lit("u"), F.col("author_id")).alias("author_id"),
    F.col("created_at"),
    F.when(F.col("in_reply_to_user_id").isNull(), False).otherwise(True).cast("int").alias("is_reply"),
    F.col("lang"),
    F.col("possibly_sensitive").cast("int").alias("is_sensitive"),
    F.col("public_metrics.like_count"),
    F.col("public_metrics.quote_count"),
    F.col("public_metrics.reply_count"),
    F.col("public_metrics.retweet_count")
    )

In [27]:
#tweets_selected.printSchema()

In [28]:
#tweets_selected.show(5, truncate=False)

In [29]:
#print('Summary of missing values:')
#tweets_selected.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in tweets_selected.columns]).show()

In [30]:
# Truncate each tweet to 25 tokens
def truncate_text(text, max_tokens=25):
    tokens = text.split()
    return " ".join(tokens[:max_tokens])

truncate_udf = F.udf(lambda x: truncate_text(x, 25), StringType())

tweets_truncated = tweets_selected.withColumn(
    "text_truncated", truncate_udf("text")
)

In [31]:
#tweets_truncated.show(1, truncate=False)

In [32]:
# Filter top 20 tweets per author
window = Window.partitionBy("author_id").orderBy(F.col("created_at").desc())
tweets_truncated = tweets_truncated.withColumn("rank", F.row_number().over(window))
tweets_filtered = tweets_truncated.filter(F.col("rank") <= 20).drop("rank")

In [33]:
tweet_features = tweets_filtered.groupBy("author_id").agg(
    F.concat_ws(" ", F.collect_list("text_truncated")).alias("tweets_last20_concatenated_text"),
    F.avg(F.col("is_reply")).alias("tweets_last20_reply_fraction"),
    F.countDistinct("lang").alias("tweets_last20_num_distinct_langs"),
    F.avg(F.col("is_sensitive")).alias("tweets_last20_sensitive_fraction"),
    F.avg(F.col("like_count")).alias("tweets_last20_avg_likes"),
#    F.avg(F.col("quote_count")).alias("tweets_last20_avg_quotes"),
#    F.avg(F.col("reply_count")).alias("tweets_last20_avg_replies"),
    F.avg(F.col("retweet_count")).alias("tweets_last20_avg_retweets")
    )

In [34]:
# Join user features with aggregated tweet features
enriched_user_features = user_features.join(
    tweet_features,
    user_features.id == tweet_features.author_id,
    how="inner"
).drop("author_id")

In [35]:
#enriched_user_features.printSchema()

In [36]:
#enriched_user_features.show(5, truncate=False)

In [46]:
text_features = enriched_user_features.select(
    F.col("id"),
    F.col("description"),
    F.col("tweets_last20_concatenated_text")
)

In [47]:
desc_doc = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("description_doc")

desc_embed = XlmRoBertaSentenceEmbeddings.pretrained("sent_xlm_roberta_base", "xx") \
    .setInputCols(["description_doc"]) \
    .setOutputCol("description_embeddings")

desc_pipeline = Pipeline(stages=[desc_doc, desc_embed])
desc_model = desc_pipeline.fit(text_features)
desc_emb = desc_model.transform(text_features).cache()

tweets_doc = DocumentAssembler() \
    .setInputCol("tweets_last20_concatenated_text") \
    .setOutputCol("tweets_doc")

tweets_embed = XlmRoBertaSentenceEmbeddings.pretrained("sent_xlm_roberta_base", "xx") \
    .setInputCols(["tweets_doc"]) \
    .setOutputCol("tweets_embeddings")

tweets_pipeline = Pipeline(stages=[tweets_doc, tweets_embed])
tweets_model = tweets_pipeline.fit(desc_emb)
tweets_emb = tweets_model.transform(desc_emb).cache()

sent_xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
[OK!]
sent_xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
[OK!]


In [48]:
tweets_emb.printSchema()

root
 |-- id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- tweets_last20_concatenated_text: string (nullable = false)
 |-- description_doc: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- description_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |

In [49]:
text_module = tweets_emb.select(
    "id",
    F.col("description_embeddings")[0]["embeddings"].alias("description_vector"),
    F.col("tweets_embeddings")[0]["embeddings"].alias("tweets_vector")
)


In [ ]:
user_module = enriched_user_features.drop(
    "description",
    "tweets_last20_concatenated_text"
    )

In [ ]:
#user_module.printSchema()

In [ ]:
# Train, validation, test split
train_frac = 0.7
val_frac = 0.2
test_frac = 0.1

train_df, val_df, test_df = user_module.randomSplit([train_frac, val_frac, test_frac], seed=42)

In [ ]:
# Standardize numerical features
numeric_cols = [
    f.name for f in train_df.schema.fields if isinstance(f.dataType, NumericType)
    ]

stats_df = train_df.select([
    F.mean(c).alias(f"{c}_mean") for c in numeric_cols
    ] + [
        F.stddev(c).alias(f"{c}_std") for c in numeric_cols
        ])

# train_df
scaled_train_df = train_df.crossJoin(stats_df)

for c in numeric_cols:
    scaled_train_df = scaled_train_df.withColumn(f"{c}_scaled",
     (F.col(c) - F.col(f"{c}_mean")) / F.col(f"{c}_std"))

# val_df
scaled_val_df = val_df.crossJoin(stats_df)

for c in numeric_cols:
    scaled_val_df = scaled_val_df.withColumn(f"{c}_scaled",
     (F.col(c) - F.col(f"{c}_mean")) / F.col(f"{c}_std"))

# test_df
scaled_test_df = test_df.crossJoin(stats_df)

for c in numeric_cols:
    scaled_test_df = scaled_test_df.withColumn(f"{c}_scaled",
     (F.col(c) - F.col(f"{c}_mean")) / F.col(f"{c}_std"))

# Select relevant cols
cols = [
    f.name for f in scaled_train_df.schema.fields
    if f.name not in numeric_cols
    and not f.name.endswith("_mean")
    and not f.name.endswith("_std")
]

scaled_train_df = scaled_train_df.select(cols)
scaled_val_df = scaled_val_df.select(cols)
scaled_test_df = scaled_test_df.select(cols)

In [ ]:
#scaled_train_df.show(5)

In [ ]:
bool_cols = [
    f.name for f in scaled_train_df.schema.fields if isinstance(f.dataType, BooleanType)
    ]

select_exprs = [
    # Cast all boolean columns to double
    F.col(c).cast("double").alias(c) for c in bool_cols
] + [
    # Cast label column: 1 if "bot", 0 if "human"
    F.when(F.col("label") == "bot", 1.0).otherwise(0.0).alias("label")
] + [
    F.col(c) for c in scaled_train_df.columns if c not in bool_cols + ["label"]
]

prepared_train_df = scaled_train_df.select(*select_exprs).cache()
prepared_val_df = scaled_val_df.select(*select_exprs).cache()
prepared_test_df = scaled_test_df.select(*select_exprs).cache()

In [44]:
path = "/content/drive/MyDrive/TwiBot-22-Processed"

prepared_train_df.write.mode("overwrite").parquet(f"{path}/prepared_train_df.parquet")
prepared_val_df.write.mode("overwrite").parquet(f"{path}/prepared_val_df.parquet")
prepared_test_df.write.mode("overwrite").parquet(f"{path}/prepared_test_df.parquet")
text_module.write.mode("overwrite").parquet(f"{path}/text_module.parquet")

In [50]:
text_module.write.mode("overwrite").parquet(f"{path}/text_module.parquet")

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.5.6-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.5.6-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 